# SQL Subqueries - Lab

## Introduction

Now that you've seen how subqueries work, it's time to get some practice writing them! Not all of the queries will require subqueries, but all will be a bit more complex and require some thought and review about aggregates, grouping, ordering, filtering, joins and subqueries. Good luck!  

## Objectives

You will be able to:

* Write subqueries to decompose complex queries

## CRM Database Schema

Once again, here's the schema for the CRM database you'll continue to practice with.

<img src="images/Database-Schema.png" width="600">

## Connect to the Database

As usual, start by importing the necessary packages and connecting to the database **data.sqlite**.

In [1]:
import pandas as pd
import sqlite3

In [2]:
conn = sqlite3.connect('data.sqlite')
cur = conn.cursor()

## Write an Equivalent Query using a Subquery

```SQL
SELECT customerNumber,
       contactLastName,
       contactFirstName
       FROM customers
       JOIN orders 
       USING(customerNumber)
       WHERE orderDate = '2003-01-31';
```

In [4]:
cur.execute("""SELECT customerNumber, contactLastName, contactFirstName
               FROM customers
               WHERE customerNumber IN (SELECT customerNumber
                                        FROM orders
                                        WHERE orderDate = '2003-01-31')""").fetchall()

[(141, 'Freyre', 'Diego ')]

## Select the Total Number of Orders for Each Product Name

Sort the results by the total number of items sold for that product.

In [6]:
cur.execute("""SELECT productName, SUM(quantityOrdered) AS sum
               FROM products
               JOIN orderdetails
               USING(productCode)
               GROUP BY productName""").fetchall()

[('18th Century Vintage Horse Carriage', 907),
 ('18th century schooner', 1011),
 ('1900s Vintage Bi-Plane', 940),
 ('1900s Vintage Tri-Plane', 1009),
 ('1903 Ford Model A', 883),
 ('1904 Buick Runabout', 990),
 ('1911 Ford Town Car', 832),
 ('1912 Ford Model T Delivery Wagon', 991),
 ('1913 Ford Model T Speedster', 1038),
 ('1917 Grand Touring Sedan', 918),
 ('1917 Maxwell Touring Car', 992),
 ('1926 Ford Fire Engine', 998),
 ('1928 British Royal Navy Airplane', 972),
 ('1928 Ford Phaeton Deluxe', 972),
 ('1928 Mercedes-Benz SSK', 880),
 ('1930 Buick Marquette Phaeton', 1074),
 ('1932 Alfa Romeo 8C2300 Spider Sport', 866),
 ('1932 Model A Ford J-Coupe', 957),
 ('1934 Ford V8 Coupe', 985),
 ('1936 Chrysler Airflow', 983),
 ('1936 Harley Davidson El Knucklehead', 945),
 ('1936 Mercedes Benz 500k Roadster', 824),
 ('1936 Mercedes-Benz 500K Special Roadster', 960),
 ('1937 Horch 930V Limousine', 884),
 ('1937 Lincoln Berline', 1111),
 ('1938 Cadillac V-16 Presidential Limousine', 955),
 (

## Select the Product Name and the  Total Number of People Who Have Ordered Each Product

Sort the results in descending order.

### A quick note on the SQL  `SELECT DISTINCT` statement:

The `SELECT DISTINCT` statement is used to return only distinct values in the specified column. In other words, it removes the duplicate values in the column from the result set.

Inside a table, a column often contains many duplicate values; and sometimes you only want to list the unique values. If you apply the `DISTINCT` clause to a column that has `NULL`, the `DISTINCT` clause will keep only one NULL and eliminates the other. In other words, the DISTINCT clause treats all `NULL` “values” as the same value.

In [9]:
cur.execute("""SELECT productName, COUNT(DISTINCT customerNumber) AS numPurchasers
               FROM products
               JOIN orderdetails
               USING(productCode)
               JOIN orders
               USING(orderNumber)
               GROUP BY productName
               ORDER BY numPurchasers DESC""").fetchall()

[('1992 Ferrari 360 Spider red', 40),
 ('Boeing X-32A JSF', 27),
 ('1972 Alfa Romeo GTA', 27),
 ('1952 Alpine Renault 1300', 27),
 ('1934 Ford V8 Coupe', 27),
 ('HMS Bounty', 26),
 ('Corsair F4U ( Bird Cage)', 26),
 ('American Airlines: MD-11S', 26),
 ('1999 Yamaha Speed Boat', 26),
 ('1996 Moto Guzzi 1100i', 26),
 ('1995 Honda Civic', 26),
 ('1980s Black Hawk Helicopter', 26),
 ('1969 Harley Davidson Ultimate Chopper', 26),
 ('1941 Chevrolet Special Deluxe Cabriolet', 26),
 ('1940 Ford Delivery Sedan', 26),
 ('1937 Horch 930V Limousine', 26),
 ('1913 Ford Model T Speedster', 26),
 ('1900s Vintage Bi-Plane', 26),
 ('18th Century Vintage Horse Carriage', 26),
 ('America West Airlines B757-200', 25),
 ('ATA: B757-300', 25),
 ('2003 Harley-Davidson Eagle Drag Bike', 25),
 ('2002 Suzuki XREO', 25),
 ('1997 BMW R 1100 S', 25),
 ('1993 Mazda RX-7', 25),
 ('1962 LanciaA Delta 16V', 25),
 ('1960 BSA Gold Star DBD34', 25),
 ('1948 Porsche 356-A Roadster', 25),
 ('1940s Ford truck', 25),
 ('1940

## Select the Employee Number, First Name, Last Name, City (of the office), and Office Code of the Employees Who Sold Products Which Have Been Ordered by Less Then 20 people.

This problem is a bit tougher. To start, think about how you might break the problem up. Be sure that your results only list each employee once.

In [12]:
cur.execute("""SELECT DISTINCT(employeeNumber), officeCode, o.city, firstName, lastName
               FROM employees e
               JOIN offices o
               USING(officeCode)
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               JOIN orders
               USING(customerNumber)
               JOIN orderdetails
               USING(orderNumber)
               WHERE productCode IN (SELECT productCode
                                            FROM products
                                            JOIN orderdetails
                                            USING(productCode)
                                            JOIN orders
                                            USING(orderNumber)
                                            GROUP BY productCode
                                            HAVING COUNT(DISTINCT customerNumber) < 20)""").fetchall()

[(1370, 4, 'Paris', 'Gerard', 'Hernandez'),
 (1501, 7, 'London', 'Larry', 'Bott'),
 (1337, 4, 'Paris', 'Loui', 'Bondur'),
 (1166, 1, 'San Francisco', 'Leslie', 'Thompson'),
 (1286, 3, 'NYC', 'Foon Yue', 'Tseng'),
 (1612, 6, 'Sydney', 'Peter', 'Marsh'),
 (1611, 6, 'Sydney', 'Andy', 'Fixter'),
 (1401, 4, 'Paris', 'Pamela', 'Castillo'),
 (1621, 5, 'Tokyo', 'Mami', 'Nishi'),
 (1323, 3, 'NYC', 'George', 'Vanauf'),
 (1165, 1, 'San Francisco', 'Leslie', 'Jennings'),
 (1702, 4, 'Paris', 'Martin', 'Gerard'),
 (1216, 2, 'Boston', 'Steve', 'Patterson'),
 (1188, 2, 'Boston', 'Julie', 'Firrelli'),
 (1504, 7, 'London', 'Barry', 'Jones')]

## Select the Employee Number, First Name, Last Name, and Number of Customers for Employees Whose Customers Have an Average Credit Limit of Over 15K

In [10]:
cur.execute("""SELECT employeeNumber, firstName, lastName, COUNT(customerNumber) AS numCustomers
               FROM employees e
               JOIN customers c
               ON e.employeeNumber = c.salesRepEmployeeNumber
               GROUP BY employeeNumber, firstName, lastName
               HAVING AVG(creditLimit) > 15000""").fetchall()

[(1165, 'Leslie', 'Jennings', 6),
 (1166, 'Leslie', 'Thompson', 6),
 (1188, 'Julie', 'Firrelli', 6),
 (1216, 'Steve', 'Patterson', 6),
 (1286, 'Foon Yue', 'Tseng', 7),
 (1323, 'George', 'Vanauf', 8),
 (1337, 'Loui', 'Bondur', 6),
 (1370, 'Gerard', 'Hernandez', 7),
 (1401, 'Pamela', 'Castillo', 10),
 (1501, 'Larry', 'Bott', 8),
 (1504, 'Barry', 'Jones', 9),
 (1611, 'Andy', 'Fixter', 5),
 (1612, 'Peter', 'Marsh', 5),
 (1621, 'Mami', 'Nishi', 5),
 (1702, 'Martin', 'Gerard', 6)]

## Summary

In this lesson, you got to practice some more complex SQL queries, some of which required subqueries. There's still plenty more SQL to be had though; hope you've been enjoying some of these puzzles!